<a href="https://colab.research.google.com/github/umairaziz999/BERT-Style-LLM-Assignment/blob/main/LLM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**
In this notebook we talk abount finetuning mbart-50 model on custom data for English to Urdu translation. We will walk through the complete process from preparing the data to feeding it into moodel.**We will be using 2 T4 GPUs for this purpose**

In [1]:
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none

# **Data Preparation**
We will use "opus100" data corpus from huggingface for this purpose. The data consist of 753913 samples of text in both English and Urdu language. I first convert the data into csv format becasuse i am comfortable with handling csv's and i also take only 6000 samples (in which 5000 is for train 1000 is for validation) of data due to the time it takes for training the model.In the following code i have done all this step by step.

In [2]:
import datasets
import csv
import os

In [3]:
dataset = datasets.load_dataset("opus100",'en-ur')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/753913 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
def save_translation_to_csv(input_dict, output_file):
    # Extract English and Urdu texts from the input dictionary
    english_text = input_dict['translation']['en']
    urdu_text = input_dict['translation']['ur']

    # Create a list of dictionaries containing the data to be saved in the CSV file
    data = [{'Urdu': urdu_text, 'English': english_text}]

    # Define the CSV file headers
    fieldnames = ['Urdu', 'English']

    # Check if the CSV file already exists
    file_exists = os.path.isfile(output_file)

    # Write the data to the CSV file
    with open(output_file, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write header only if the file is newly created
        if not file_exists:
            writer.writeheader()

        # Write the data row
        writer.writerows(data)


In [5]:
output_csv_file = 'translation.csv'
for n in range(6000):
   input_text=dataset['train'][n]
   save_translation_to_csv(input_text, output_csv_file)

# **Loading Data**
Now we load the data and prepare it to feed into model for that i have created 5 different style of prompts. By different style of prompts i mean i have defined 5 different ways in which the question is asked to the model and the answer is given by the model. As we know that every user try different ways to ask the same think so thats why i am defining 5 different ways you can change the prompts also increase the ways according to the portion of data you use

In [6]:
from datasets import concatenate_datasets,Dataset

In [7]:
import pandas as pd

In [8]:
dataset_file1 = '/content/translation.csv'
dataset1 = pd.read_csv(dataset_file1)


In [9]:
dataset1

,Urdu,English
0,اورجب ہم نے موسیٰ سے چالیس رات کا وعدہ کیا پھر...,"Yet, remember, as We communed with Moses for f..."
1,"بجز تیرے اُن بندوں کے جنہیں تو نے خالص کر لیا ہے""","Except, among them, Your chosen servants."""
2,(اے پیغمبر لوگوں سے) کہہ دو کہ میرے پاس وحی آئ...,"Say, (O Prophet), it was revealed to me that a..."
3,ہم عنقریب اس کی ناک پر داغ لگائیں گے,Soon We shall brand him on his snout.
4,اور جو لوگ مومن مردوں اور عورتوں کو بے قصور اذ...,And those who unnecessarily harass Muslim men ...
...,...,...
5995,اور ہم نے انہیں بانٹ دیا بارہ قبیلے گروہ گروہ،...,We divided them into twelve tribal communities...
5996,جو تمہارا بھی پروردگار ہے اور تمہارے پہلے والے...,who is your Lord and the Lord of your forefath...
5997,تاکہ وہ اس کے پھل کھائیں اور اسے اُن کے ہاتھوں...,So that they may eat of its fruit; yet it was ...
5998,اتنے میں شہر کے دُور دراز گوشے سے ایک شخص دوڑت...,"Then there came running, from the farthest par..."


In [10]:
train_dataset=dataset1[:500]
val_dataset=dataset1[500:]

In [11]:
val_dataset.shape

(5500, 2)

Here we are dividing the 5000 train samples into 5 parts 1000 samples for a single type of prompt

In [12]:
data1=train_dataset[:100]
data2=train_dataset[100:200]
data3=train_dataset[200:300]
data4=train_dataset[300:400]
data5=train_dataset[400:500]

Here i am creating a different dataset object and filling it by creating a prompt along with the data.Here engtext colomn is the source colomn and urdutext colomn is the target colomn

In [13]:


train_dataset1 = pd.DataFrame()
train_dataset1['engtext'] ='Can you translate "' + data1['English'] + '" into urdu ?'
train_dataset1['urdutext'] = 'Yes sure here is the urdu translation "' + data1['Urdu'] + '".'

train_dataset2 = pd.DataFrame()
train_dataset2['engtext'] = 'Please Translate "' + data2['English'] + '" to urdu.'
train_dataset2['urdutext'] = 'Okey the translation into urdu is "' + data2['Urdu'] + '". I hope this will help.'

train_dataset3 = pd.DataFrame()
train_dataset3['engtext'] = 'Please Translate "' + data3['English'] + '" to urdu.'
train_dataset3['urdutext'] = "Yeah sure i can help you with that here is the translation in Urdu : " + data3['Urdu'] + '.'

train_dataset4 = pd.DataFrame()
train_dataset4['engtext'] = 'Can you translate "' + data4['English'] + '" into urdu ?'
train_dataset4['urdutext'] = 'Yes, the translation of "' + data4['English'] + '" to Urdu is : "' + data4['Urdu'] + '".'

train_dataset5 = pd.DataFrame()
train_dataset5['engtext'] = 'Translate English text to Urdu. Here is the text :"' + data5['English'] + '".'
train_dataset5['urdutext'] = 'Sure, the translation of your given text to Urdu is : "' + data5['Urdu'] + '".'

appended_dataframe = pd.concat([train_dataset1, train_dataset2, train_dataset3, train_dataset4, train_dataset5], ignore_index=True)

train_data = Dataset.from_pandas(appended_dataframe)
train_data

Dataset({
    features: ['engtext', 'urdutext'],
    num_rows: 500
})

In [14]:
train_data['engtext'][400]

'Translate English text to Urdu. Here is the text :"There will profit you neither your kindred nor your children on the Day of Resurrection. He shall decide between; and Allah is of that which ye you work Beholder.".'

In [15]:
train_data['urdutext'][400]

'Sure, the translation of your given text to Urdu is : "قیامت کے دن نہ تمہاری رشتہ داریاں کسی کام آئیں گی نہ تمہاری اولاد اُس روز اللہ تمہارے درمیان جدائی ڈال دے گا، اور و ہی تمہارے اعمال کا دیکھنے والا ہے".'

In [16]:
data1=val_dataset[:100]
data2=val_dataset[100:200]
data3=val_dataset[200:300]
data4=val_dataset[300:400]
data5=val_dataset[400:500]

In [17]:
import pandas as pd
from datasets import Dataset


val_dataset1 = pd.DataFrame()
val_dataset1['engtext'] ='Can you translate "' + data1['English'] + '" into urdu ?'
val_dataset1['urdutext'] = 'Yes sure here is the urdu translation "' + data1['Urdu'] + '".'

val_dataset2 = pd.DataFrame()
val_dataset2['engtext'] = 'Please Translate "' + data2['English'] + '" to urdu.'
val_dataset2['urdutext'] = 'Okey the translation into urdu is "' + data2['Urdu'] + '". I hope this will help.'

val_dataset3 = pd.DataFrame()
val_dataset3['engtext'] = 'Please Translate "' + data3['English'] + '" to urdu.'
val_dataset3['urdutext'] = "Yeah sure i can help you with that here is the translation in Urdu : " + data3['Urdu'] + '.'

val_dataset4 = pd.DataFrame()
val_dataset4['engtext'] = 'Can you translate "' + data4['English'] + '" into urdu ?'
val_dataset4['urdutext'] = 'Yes, the translation of "' + data4['English'] + '" to Urdu is : "' + data4['Urdu'] + '".'

val_dataset5 = pd.DataFrame()
val_dataset5['engtext'] = 'Translate English text to Urdu. Here is the text :"' + data5['English'] + '".'
val_dataset5['urdutext'] = 'Sure, the translation of your given text to Urdu is : "' + data5['Urdu'] + '".'

appended_dataframe = pd.concat([val_dataset1, val_dataset2, val_dataset3, val_dataset4, val_dataset5], ignore_index=True)

val_data = Dataset.from_pandas(appended_dataframe)
val_data


Dataset({
    features: ['engtext', 'urdutext'],
    num_rows: 500
})

In [18]:
val_data['engtext'][400]

'Translate English text to Urdu. Here is the text :"And the Jews said, “Allah’s hand is tied"; may their hands be tied – and they are accursed for saying so! In fact, both His hands* are free, He bestows upon whomever He wills; and O dear Prophet, this Book which has been sent down upon you from your Lord will cause many of them to advance in their rebellion and disbelief; and We have instilled enmity and hatred between them till the Day of Resurrection; whenever they ignite the flame of war, Allah extinguishes it, and they strive to create chaos in the land; and Allah does not love the mischievous. (* This is a metaphor used to express Allah’s power).".'

In [19]:
val_data['urdutext'][400]

'Sure, the translation of your given text to Urdu is : "اور یہودی بولے اللہ کا ہاتھ بندھا ہوا ہے ان کے ہاتھ باندھے جائیں اور ان پر اس کہنے سے لعنت ہے بلکہ اس کے ہاتھ کشادہ ہیں عطا فرماتا ہے جیسے چاہے اور اے محبوب! یہ جو تمہاری طرف تمہارے رب کے پاس سے اترا اس سے ان میں بہتوں کو شرارت اور کفر میں ترقی ہوگی اور ان میں ہم نے قیامت تک آپس میں دشمنی اور بیر ڈال دیا جب کبھی لڑائی کی آگ بھڑکاتے ہیں اللہ اسے بجھا دیتا ہے اور زمین میں فساد کے لیے دوڑتے پھرتے ہیں، اور اللہ فسادیوں کو نہیں چاہتا،".'

# **Loading the model**

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer,MBartForConditionalGeneration

model_ckpt = "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = MBartForConditionalGeneration.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

Now we are going to create a function that takes the text data and convert it into input_ids and labels because as we know computer only understand numbers not text so we need to convert text into numbers for that we are using mbart tokenizer. Here max_length is set 1024 because that is the maximam tokens of string mbart can process at a time

In [21]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["engtext"], max_length=1024, padding="max_length",truncation=True)

    #with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch["urdutext"], max_length=1024, padding="max_length", truncation=True)

    return {"input_ids": input_encodings["input_ids"],
           "attention_mask": input_encodings["attention_mask"],
           "labels": target_encodings["input_ids"]}

train_dataset_tf = train_data.map(convert_examples_to_features, batched=True, remove_columns=["engtext","urdutext"])
val_dataset_tf = val_data.map(convert_examples_to_features, batched=True, remove_columns=["engtext","urdutext"])

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [22]:
train_dataset_tf

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})

In [23]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [24]:
from huggingface_hub import notebook_login

notebook_login()

In [31]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer,TrainingArguments, Trainer

training_args = Seq2SeqTrainingArguments(output_dir='mbartTrans', num_train_epochs=1,
                                  per_device_train_batch_size=1, per_device_eval_batch_size=1,
                                evaluation_strategy='steps',save_strategy='no',eval_steps=10,logging_steps=1,
                                  weight_decay=0.01, push_to_hub=False,
                           learning_rate=2e-5,optim="adafactor")

In [32]:
import gc
gc.collect()

1930

In [ ]:
trainer = Seq2SeqTrainer(model=model, args=training_args, tokenizer=tokenizer,
                  data_collator=seq2seq_data_collator,
                  train_dataset=train_dataset_tf,
                        eval_dataset=val_dataset_tf)

trainer.train()

Step,Training Loss,Validation Loss
10,0.037500,0.137477
20,0.050300,0.140807
30,0.074500,0.141515
40,0.073000,0.146567
50,0.030700,0.142765
60,0.013500,0.143882
70,0.125100,0.141991
80,0.098100,0.142086
90,0.058600,0.176781
100,0.101900,0.145646


In [ ]:
trainer.save_model("mbartfortranslation5000")

In [ ]:
import shutil
shutil.make_archive('mbartfortranslation', 'zip', '/kaggle/working/mbartfortranslation5000')

> # **Testing the model on custom prompts**

Note: Restart and clear cell output to release gpus memory because we are loading the model on gpu for generating faster responces. Save the notebook sos that you can not lose your work

In [ ]:
from transformers import MBartForConditionalGeneration, AutoTokenizer,pipeline

In [ ]:
model_ckpt = "/kaggle/working/bloomzfortran"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForCausalLM.from_pretrained("/kaggle/working/bloomzfortran").to('cuda:0').half()

In [ ]:
prompt1='Please Translate "How are you." to urdu.'
prompt1

In [ ]:
generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer,device='cuda:0',do_sample=False)

In [ ]:
result = generator(prompt, max_length=1024)

In [ ]:
result